# Automatic Translation

## Make workspace

In [ ]:
# Make directories if they don't exist
!mkdir -p datasets
!mkdir -p plot
!mkdir -p model

## Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

## Download datasets

Translation dataset used in original paper can be found [here](https://www.tensorflow.org/datasets/catalog/wmt14_translate)

In [ ]:
# Download english language word list
if not os.path.exists('datasets/source_dictionary.txt'):
    !cd datasets && curl -o source_dictionary.txt https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt

In [ ]:
# Download french language word list
if not os.path.exists('datasets/target_dictionary.tsv'):
    !cd datasets && curl -O http://www.lexique.org/databases/Lexique383/Lexique383.zip
    !cd datasets && unzip Lexique383.zip
    !mv datasets/Lexique383.tsv datasets/target_dictionary.tsv

In [ ]:
# Download english-to-french translation dataset
if not os.path.exists('datasets/translation.txt'):
    !cd datasets && curl -O http://www.manythings.org/anki/fra-eng.zip
    !cd datasets && unzip fra-eng.zip fra.txt
    !mv datasets/fra.txt datasets/translation.txt

In [ ]:
# Download english word embeddings
if not os.path.exists('datasets/glove.6B.100d.txt'):
    !cd datasets && curl -LO http://nlp.stanford.edu/data/glove.6B.zip
    !cd datasets && unzip glove.6B.zip glove.6B.100d.txt

## Preprocessing

Sentences from the source and destination languages are converted to token sequences and padded to the same length. Source sentences are padded on the left and destination sentences on the right. Special tokens representing words not in dicionary (\<unk\>), start of sequence (\<sos\>) and end of sequence (\<eos\>) are added to the destination sentences. There are two generated destination sentences, one for decoder input (which has the start token appended), and one for decoder ouput (which has the end token appended). Source language sequences are reversed for better results.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

### English word list

In [ ]:
# Word list for source language
source_words = []

# Load source language word list and convert characters to lower
with open('datasets/source_dictionary.txt') as f:
    source_words = f.read().lower().split('\n')
print(source_words[:20])

# Make source language tokenizer. UNK token is 1
source_tokenizer = Tokenizer(num_words=len(source_words), oov_token=1)
source_tokenizer.fit_on_texts(source_words)

### French word list

In [ ]:
# Word list for target language
target_words = []

# Load target language word list and convert characters to lower
df = pd.read_csv('datasets/target_dictionary.tsv', sep='\t', keep_default_na=False)['ortho'].tolist()
target_words = [w.lower() for w in df]
print(target_words[:20])

# Make target language tokenizer. UNK token is 1
target_tokenizer = Tokenizer(num_words=len(target_words), oov_token=1)
target_tokenizer.fit_on_texts(target_words)

### English-to-French translation dataset

In [ ]:
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'
UNK_TOKEN = '<unk>'

In [ ]:
source_sentences = []
target_sentences_input = []
target_sentences_output = []

lines = []
with open('datasets/translation.txt') as f:
    lines = f.read().split('\n')

NUM_LINES = 20_000
for line in lines[:NUM_LINES]:
    input_sentence, target_sentence, _ = line.split('\t')
    source_sentences.append(input_sentence)
    target_sentences_input.append(SOS_TOKEN + ' ' + target_sentence)
    target_sentences_output.append(target_sentence + ' ' + EOS_TOKEN)

In [ ]:
MAX_NUM_SOURCE_WORDS = 50_000
MAX_NUM_TARGET_WORDS = 50_000

In [ ]:
# Make source language tokenizer
source_tokenizer = Tokenizer(num_words=MAX_NUM_SOURCE_WORDS)
source_tokenizer.fit_on_texts(source_sentences)
source_dict = source_tokenizer.word_index
NUM_SOURCE_WORDS = len(source_dict)+1

# Make destination language tokenizer
# Don't filter '<' and '>' because they're used for SOS and EOS tokens
target_tokenizer = Tokenizer(num_words=MAX_NUM_TARGET_WORDS, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
target_tokenizer.fit_on_texts(target_sentences_input + target_sentences_output)
target_dict = target_tokenizer.word_index
NUM_TARGET_WORDS = len(target_dict)+1

In [ ]:
print(len(source_dict))
print(len(target_dict))
print(source_dict)
print(target_dict)

In [ ]:
# Tokenize sentences to sequences
source_sequences = source_tokenizer.texts_to_sequences(source_sentences)
target_sequences_input = target_tokenizer.texts_to_sequences(target_sentences_input)
target_sequences_output = target_tokenizer.texts_to_sequences(target_sentences_output)

# Reverse source sequnces
for sequence in source_sequences:
    sequence.reverse()

max_source_len = max([len(sequence) for sequence in source_sequences])
max_target_len = max([len(sequence) for sequence in target_sequences_output])

encoder_input_sequences = pad_sequences(source_sequences, maxlen=max_source_len, padding='pre')
decoder_input_sequences = pad_sequences(target_sequences_input, maxlen=max_target_len, padding='post')
decoder_output_sequences = pad_sequences(target_sequences_output, maxlen=max_target_len, padding='post')

In [ ]:
print(max_source_len)
print(max_target_len)

In [ ]:
for i in range(0, 10):
    print(source_sentences[i])
    print(str(encoder_input_sequences[i]))
    print(target_sentences_input[i])
    print(str(decoder_input_sequences[i]))

## Model Building

The most common model for neural machine translation (NMT) is the sequence-to-sequence model which is a combination of recurrent neural network and encoder-decoder architectures. The enocder maps variable-length input sequence to a fixed-length representation, and the decoder uses that representation to generate an output sequence one word at a time.

This type of model is used by Google for its translation service (https://arxiv.org/pdf/1609.08144.pdf, https://ai.googleblog.com/2016/09/a-neural-network-for-machine.html).

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense 

In [ ]:
# Number of tokens is expected to be 1 greater than the number of words
n_encoder_tokens = len(source_dict)+1
n_decoder_tokens = len(target_dict)+1

EMBEDDING_DIM = 100
LSTM_DIM = 256

### Embedding layer

Keras documentation: https://keras.io/api/layers/core_layers/embedding/

Turns positive integers (indexes) into dense vectors of fixed size.

e.g. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

This layer can only be used as the first layer in a model.

Arguments:
- **input_dim**: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
- **output_dim**: Integer. Dimension of the dense embedding.

### LSTM layer

Keras documentation: https://keras.io/api/layers/recurrent_layers/lstm/

Long Short-Term Memory layer

Arguments:
- **units**: Positive integer, dimensionality of the output space.
- **kernel_initializer**: Initializer for the kernel weights matrix
- **return_sequences**: Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False.
- **return_state**: Boolean. Whether to return the last state in addition to the output. Default: False.

return_state returns the lstm output, last hidden state and last cell state and return_sequences returns hidden states for each step as the main output. Both flags can be used at the same time and will return all hidden states, last hidden state and last cell state.

For more information on return_sequences and return_state read [this](https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/)

### Layer initializers

Keras documentation: https://keras.io/api/layers/initializers/

Initializers define the way to set the initial random weights of Keras layers.

Some of the available initializers:
- RandomNormal
- RandomUniform
- Zeros
- Ones

### GloVe Embeddings

Instead of learning the embeddings for english words during training, a pre-trained model called [GloVe](https://nlp.stanford.edu/projects/glove/) will be used. It has been trained on 6 billion words from [Wikipedia 2014](https://dumps.wikimedia.org/enwiki/latest/) and [Gigaworld 5](https://catalog.ldc.upenn.edu/LDC2011T07). The french word embeddings will be learned during training.

In [ ]:
embeddings_dict = {}
with open('datasets/glove.6B.100d.txt', encoding="utf8") as glove_file:
    for line in glove_file:
        record = line.split()
        word = record[0]
        word_embedding = np.asarray(record[1:], dtype='float32')
        embeddings_dict[word] = word_embedding

In [ ]:
embedding_matrix = np.zeros((n_encoder_tokens, EMBEDDING_DIM))
for word, index in source_dict.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
print(embeddings_dict["hello"])

### Encoder

In [ ]:
encoder_inputs = Input(shape=(max_source_len,))
encoder_embedding = Embedding(n_encoder_tokens, EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_source_len)
encoder_lstm = LSTM(
    LSTM_DIM,
    # kernel_initializer = tf.keras.initializers.RandomUniform(minval=-0.08, maxval=0.08, seed=None),
    return_state=True
)

encoder = encoder_embedding(encoder_inputs)
encoder, state_h, state_c = encoder_lstm(encoder)

encoder_states = [state_h, state_c]

### Decoder

In [ ]:
decoder_inputs = Input(shape=(max_target_len,))
decoder_embedding = Embedding(n_decoder_tokens, EMBEDDING_DIM)
decoder_lstm = LSTM(
    LSTM_DIM, 
    # kernel_initializer = tf.keras.initializers.RandomUniform(minval=-0.08, maxval=0.08, seed=None),
    return_sequences=True,
    return_state=True
)
decoder_dense = Dense(n_decoder_tokens, activation='softmax')

decoder = decoder_embedding(decoder_inputs)
decoder, _, _ = decoder_lstm(decoder, initial_state=encoder_states)
decoder = decoder_dense(decoder)

### Model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder)

# It is possible to use beam search during training by definig a custom loss function
# https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618

# TODO Make adaptive learning rate using keras.callbacks.LearningRateScheduler
# https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/
# TODO Use SGD optimizer with no momentum, start rate 0.7
# https://keras.io/api/optimizers/sgd/
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

### Plot the model

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='plot/model.png', show_shapes=True, show_layer_names=True)

## Model Training

In [ ]:
print(encoder_input_sequences.shape)
print(decoder_input_sequences.shape)
print(decoder_output_sequences.shape)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

In [ ]:
from tensorflow.keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, encoder_input, decoder_input, decoder_output, n_classes, batch_size=32, shuffle=True):
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.decoder_output = decoder_output
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    # Number of batches per epoch
    def __len__(self):
        return int(np.floor(len(self.encoder_input) / self.batch_size))

    # Generate one batch
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = [self.encoder_input[indexes], self.decoder_input[indexes]]
        # Decoder outputs have to be one-hot-encoded
        y = np.asarray([to_categorical(self.decoder_output[index], num_classes=self.n_classes) for index in indexes])
        return X, y

    # Update indexes for next epoch
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.encoder_input))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [ ]:
indexes = np.arange(len(encoder_input_sequences))
np.random.shuffle(indexes)

validation_size = int(np.floor(len(encoder_input_sequences) * VALIDATION_SPLIT))

train_indexes = indexes[:-validation_size]
validation_indexes = indexes[-validation_size:]

training_generator = DataGenerator(
    encoder_input_sequences[train_indexes],
    decoder_input_sequences[train_indexes],
    decoder_output_sequences[train_indexes],
    n_decoder_tokens,
    BATCH_SIZE
)

validation_generator = DataGenerator(
    encoder_input_sequences[validation_indexes],
    decoder_input_sequences[validation_indexes],
    decoder_output_sequences[validation_indexes],
    n_decoder_tokens,
    BATCH_SIZE
)

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model/checkpoint',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)
history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint_callback]
)
model.save(f'model/tranlator_{NUM_LINES}_{EPOCHS}_{BATCH_SIZE}_{VALIDATION_SPLIT}_{time.strftime("%Y%m%d_%H%M%S")}')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.epoch, history.history['accuracy'])
plt.plot(history.epoch, history.history['val_accuracy'])
plt.legend(['Training accuracy', 'Validation accuracy'])

### Save / Load

In [ ]:
model.save_weights('model/translator_weights')

In [ ]:
model.save('model/translator')

In [ ]:
model.load_weights('model/translator_weights')

In [ ]:
keras.models.load_model('model/translator')

## Inference

### Build the model

To predict outputs a separate model is needed because the previous decoder output has to be passed in as an input for the next word

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_h = Input(shape=(LSTM_DIM,))
decoder_state_c = Input(shape=(LSTM_DIM,))
decoder_states = [decoder_state_h, decoder_state_c]

decoder_input_word = Input(shape=(1,))

inference_decoder = decoder_embedding(decoder_input_word)
inference_decoder, state_h, state_c = decoder_lstm(inference_decoder, initial_state=decoder_states)
inference_decoder = decoder_dense(inference_decoder)

inference_states = [state_h, state_c]

decoder_model = Model(
    [decoder_input_word] + decoder_states,
    [inference_decoder] + inference_states
)

In [ ]:
encoder_model.summary()
decoder_model.summary()

In [ ]:
plot_model(decoder_model, to_file='plot/inference_model.png', show_shapes=True, show_layer_names=True)

### Greedy Search

In [ ]:
reverse_source_dict = { v:k for k,v in source_dict.items() }
reverse_target_dict = { v:k for k,v in target_dict.items() }

In [ ]:
def sentence_to_sequence(sentence):
    sequence = source_tokenizer.texts_to_sequences([sentence])
    sequence.reverse()
    sequence = pad_sequences([sequence], maxlen=max_source_len, padding='pre')
    return sequence

In [ ]:
def sequence_to_sentence(sequence):
    SOS_ID = target_dict[SOS_TOKEN]
    EOS_ID = target_dict[EOS_TOKEN]
    words = []
    for i in sequence:
        if i == SOS_ID:
            continue
        elif i == EOS_ID:
            break
        elif i > 0:
            words.append(reverse_target_dict[i])
        else:
            words.append(UNK_TOKEN)
    return ' '.join(words)

In [ ]:
def predict_sequence(source_sequence):
    # Predict the encoder result directly
    current_state = encoder_model.predict(source_sequence)
    
    # Predict ouput words one at a time until <eos> token or max_target_len
    EOS_ID = target_dict[EOS_TOKEN]
    
    # Decoder_model expects a tensor as input
    decoder_input = np.zeros((1, 1))
    decoder_input[0, 0] = target_dict[SOS_TOKEN]
    target_sequence = []
    
    for _ in range(max_target_len):
        dense_outputs, state_h, state_c = decoder_model.predict([decoder_input] + current_state)
        token = np.argmax(dense_outputs[0, 0, :])
        if token == EOS_ID:
            break
        else:
            target_sequence.append(token)
        
        # Replace decoder inputs to last generated token and states
        decoder_input[0, 0] = token
        current_state = [state_h, state_c]
    return target_sequence

def predict(source_sequence):
    target_sequence = predict_sequence(source_sequence)
    return sequence_to_sentence(target_sequence)

### Beam Search

The beam search algorithm is used as an improvement to the greedy search algorithm for determining the next word in the sequence. It works by taking the k best options at each step and then using those for all next steps. The best options are evalueted by maximizing the average log probability of each word is the output. For k=1 this is equivalent to the greedy algorithm. Increasing the k value gives better results but also increases processing time exponentially.

In [ ]:
def beam_search_predict_sequence(source_sequence, k=1):
    encoder_output_state = encoder_model.predict([source_sequence])
    decoder_input = np.zeros((1, 1))
    
    # (log(1), initial_sos_token, current_state)
    k_beam = [(0, [target_dict[SOS_TOKEN]], encoder_output_state)]

    EOS_ID = target_dict[EOS_TOKEN]
    for i in range(max_target_len):
        all_k_beams = []
        for prob, predictions, state in k_beam:
            if predictions[-1] == EOS_ID:
                all_k_beams.append((prob, predictions, state))
                continue
            
            decoder_input[0,0] = predictions[-1]
            dense_outputs, state_h, state_c = decoder_model.predict([decoder_input] + state)
            
            # Get indices of top k predictions (last k when sorted)
            top_k = dense_outputs[0,0].argsort()[-k:]

            # Add to all possible candidates for k-beams
            all_k_beams += [
                (
                    # We subtract the log because it's negative
                    # Same as adding but sorting in reverse
                    prob - np.log(dense_outputs[0,0,next_word]),
                    # Append next word to the copy of existing list
                    list(predictions)+[next_word],
                    # Set state to new decoder state
                    [state_h, state_c]
                )
                for next_word in top_k
            ]

        # Get k best tuples sorted by score/length
        k_beam = sorted(all_k_beams, key = lambda t: t[0]/len(t[1]))[:k]

    # Return best sequence
    return k_beam[0][1]

def beam_search_predict(source_sequence, k=1):
    target_sequence = beam_search_predict_sequence(source_sequence, k)
    return sequence_to_sentence(target_sequence)

### Testing

In [ ]:
i = 14261
print(source_sentences[i])
print(target_sentences_output[i])

sample_input_sequences = [
    encoder_input_sequences[i:i+1]
]

for seq in sample_input_sequences:
    print(predict(seq))
    print(beam_search_predict(seq, 2))

## Analysis

### Ground truth comparison

In [ ]:
# TODO

### 2D Encoder hidden state PCA

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
# TODO Predict state for some sentences
i = 0
n = 5

# Sample_sentences = [ ]
sample_sentences = source_sentences[i:i+n+1]

# TODO make make_sequence function
source_sequences = source_tokenizer.texts_to_sequences(sample_sentences)
# Reverse source sequnces
for sequence in source_sequences:
    sequence.reverse()
sample_sequences = pad_sequences(source_sequences, maxlen=max_source_len, padding='pre')

sample_inputs = encoder_model.predict(sample_sequences)[0]

# print(sample_sentences)
# print(sample_inputs)

pca = PCA(n_components=2)
pca.fit(sample_inputs)
pca_inputs = pca.transform(sample_inputs)

plt.scatter(pca_inputs[:,0], pca_inputs[:,1])
for i in range(len(sample_sentences)):
    plt.text(pca_inputs[i,0], pca_inputs[i, 1], sample_sentences[i])

### BLEU score

BLEU (Bilingual Evaluation Understudy) is a score for comparing machine-translated text to one or more reference translations made by a human. Scoring usuall works on indiviual sentences and is then averaged on the entire text. Scores have a value between 0 and 1.

For more info:
- https://en.wikipedia.org/wiki/BLEU
- https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
- https://towardsdatascience.com/bleu-bilingual-evaluation-understudy-2b4eab9bcfd1

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# References

- https://arxiv.org/abs/1409.3215
- https://arxiv.org/abs/1406.1078
- https://arxiv.org/abs/1609.08144
- https://keras.io/examples/nlp/lstm_seq2seq/
- https://nlp.stanford.edu/projects/glove/